# Influxdb notebook for SDS-sensenet

Before getting started, make sure to install the 'InfluxDB-client' library:

`pip3 install git+https://github.com/influxdata/influxdb-client-python.git`

In [1]:
import requests
import pandas as pd
import influxdb

from datetime import datetime
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

Modify 'hostname', 'user' and 'passwd' with your credentials, not mine :0)

In [2]:
hostname = ''
user = '' 
passwd = ''
client = influxdb.DataFrameClient(hostname, 8086, ssl=True, username=user, password=passwd)

We are now connected, so the **fun** will start. First, let's query for the available DBs:

In [3]:
client.query("show databases;")

ResultSet({'('databases', None)': [{'name': 'sample_database'}, {'name': '_internal'}, {'name': 'sensornet_db'}, {'name': 'sensenet_coords'}]})

In [ ]:
coords = client.query("select * from coords", database="sensenet_coords")
df_coords = coords['coords']
df_coords = df_coords.reset_index(drop=True)
df_coords

In [5]:
client.query("show measurements", database="sensornet_db")

ResultSet({'('measurements', None)': [{'name': 'mqtt_consumer'}, {'name': 'sds'}, {'name': 'sds-sensenet'}, {'name': 'sensenet-test'}, {'name': 'sensenet-test-proto'}, {'name': 'user_coords'}]})

In [6]:
complete_df = client.query("select * from \"sensenet-test\"", database="sensornet_db")
df = complete_df['sensenet-test']
df[['dev_id','pm25', 'pm10', 'co2', 'temp', 'humidity', 'gtw_id', 'rssi', 'snr']].head()

,dev_id,pm25,pm10,co2,temp,humidity,gtw_id,rssi,snr
2020-02-03 00:36:03.779007152+00:00,sbox6,0,0,144,12.52,62.89,sds-gateway-3,-14,9.00
2020-02-03 00:39:38.171639705+00:00,sbox11,0,0,567,19.44,58.00,sds-gateway-1,-61,11.25
2020-02-03 00:39:57.440747505+00:00,sbox10,1,1,491,16.99,51.67,center-for-civic-innovation,-69,10.50
2020-02-03 00:40:39.755830804+00:00,sbox3,0,0,1247,19.29,37.97,sds-gateway-1,-60,12.25
2020-02-03 00:41:41.145261920+00:00,sbox4,0,0,365,17.36,43.69,sds-alex-apt,-45,8.50


Now, if we want to query for just a certain portion of the dataset or a particular time slice: (remember the time is in [RFC3339](https://docs.influxdata.com/influxdb/v1.7/query_language/data_exploration/#specify-a-time-range-with-epoch-timestamps))

In [7]:
timeslice = client.query("select \"co2\" FROM \"sensenet-test\" \
                         WHERE \"dev_id\" = 'sbox11' \
                         AND time >= '2020-02-01' AND time <= '2020-02-20 12:30:00'", \
                         database="sensornet_db")
timeslice

defaultdict(list, {'sensenet-test':                                       co2
             2020-02-03 00:39:38.171639705+00:00   567
             2020-02-03 00:54:38.936284266+00:00   565
             2020-02-03 01:09:37.977429403+00:00   569
             2020-02-03 01:24:38.046949854+00:00   600
             2020-02-03 01:39:37.975148414+00:00  4781
             2020-02-03 01:54:38.045715460+00:00  4420
             2020-02-03 02:09:37.960825143+00:00  3761
             2020-02-03 02:24:38.006061482+00:00  3590
             2020-02-03 02:39:37.950974992+00:00  2886
             2020-02-03 02:54:37.901787132+00:00  2468
             2020-02-03 03:09:37.938877330+00:00  2129
             2020-02-03 03:24:37.889133783+00:00  1852
             2020-02-03 03:39:37.832734317+00:00  1589
             2020-02-03 03:54:37.869381204+00:00  1395
             2020-02-03 04:09:37.822839993+00:00  1324
             2020-02-03 04:24:37.859874065+00:00  1206
             2020-02-03 04:39:37.808492784

Or, to query for all the sensorboxes with regular expressions + more intuitive time slicers!

One could use now() - 1h(our), 1w(eek), etc.

In [8]:
timeslice2 = client.query("select * FROM \"sensenet-test\" \
                         WHERE \"dev_id\" =~ /./ \
                         AND time > now() - 6w", \
                         database="sensornet_db")
ts_dt = timeslice2['sensenet-test']
ts_dt[['pm25', 'pm10', 'dev_id']].head()

,pm25,pm10,dev_id
2020-02-03 00:36:03.779007152+00:00,0,0,sbox6
2020-02-03 00:39:38.171639705+00:00,0,0,sbox11
2020-02-03 00:39:57.440747505+00:00,1,1,sbox10
2020-02-03 00:40:39.755830804+00:00,0,0,sbox3
2020-02-03 00:41:41.145261920+00:00,0,0,sbox4


You might want to filter out indoor from outdoor sensors with a regular expression: for sbox-[0-9]i sensors:

In [9]:
timeslice3 = client.query("select * FROM \"sensenet-test\" \
                              WHERE \"dev_id\" =~ /sbox-/ \
                              AND time >= now() - 1w", \
                              database="sensornet_db")
ts3_df = timeslice3['sensenet-test']
ts3_df[['pm25', 'pm10', 'dev_id']].head()

,pm25,pm10,dev_id
2020-02-23 21:58:44.793694120+00:00,2,2,sbox-i0
2020-02-23 21:59:44.693634360+00:00,1,1,sbox-i0
2020-02-23 22:00:44.676907135+00:00,1,1,sbox-i0
2020-02-23 22:01:44.731460038+00:00,1,1,sbox-i0
2020-02-23 22:02:44.640079917+00:00,1,1,sbox-i0


Now, it is time for some Panda magic: concatenate the tables with location, volunteer, and sensor values for all of the outdoor sensors (except for **!~** indoor sensors):

In [10]:
timeslice4 = client.query("select * FROM \"sensenet-test\" \
                              WHERE \"dev_id\" !~ /sbox-/ \
                              AND time >= now() - 30m", \
                              database="sensornet_db")

# get the dataframe with columns we want:
ts4_df = timeslice4['sensenet-test']
ts4_df = ts4_df[['pm25', 'pm10', 'dev_id']]
ts4_df

,pm25,pm10,dev_id
2020-02-24 22:37:28.492574821+00:00,0,0,sbox8
2020-02-24 22:41:07.645768024+00:00,0,1,sbox7
2020-02-24 22:43:09.935985245+00:00,10,10,sbox5
2020-02-24 22:47:54.753220099+00:00,0,0,sbox11
2020-02-24 22:49:53.431506584+00:00,0,0,sbox3
2020-02-24 22:50:14.077774922+00:00,1,1,sbox9
2020-02-24 22:51:13.765706481+00:00,2,2,sbox10
2020-02-24 22:52:28.431596212+00:00,0,0,sbox8
2020-02-24 22:56:07.695811926+00:00,0,1,sbox7
2020-02-24 22:58:09.923574724+00:00,10,10,sbox5


We have PM values for all sensors within a particular time slice (30 minutes). Now, we need to perform the merging with the table with geographic coordinates. Pandas to the rescue!

In [ ]:
# new variable to manipulate LAT/LONG table:
new_coords = df_coords

# shift index to the left, so we do not lose timestamps during merge:
ts4_df = ts4_df.reset_index(level=0).rename(columns={'index': 'timestamp'})

# merge dataframes using dev_id from two different columns, 'box' and 'dev_id':
merged_df = pd.merge(ts4_df, new_coords[['box', 'latitude', 'longitude', 'uid']], \
                     left_on='dev_id', \
                     right_on='box',   \
                     how='left')

merged_df
final_result = merged_df.drop(columns='box')
final_result

Influx has an unusual syntax... so when querying for fields you must use single quotes ' ' (otherwise the queries fail). We also have to escape the double quotes **inside** the query we want to run. To be happy, be syntax-safe out there! The world is a troubled spot.

Enjoy environmental monitoring!

`-- sign-off: LFRM, 02-24-2019`